In [1]:
import pandas as pd
import datetime as dt
import math
import os

# 설문지&수면 데이터와 사용자 정보 데이터 병합

In [10]:
# 사용자 정보 데이터 
data_info=pd.read_csv('./data/user_info_2020.csv')
data_info.sort_values('userId',inplace=True)
data_info.reset_index(inplace=True)
data_info.drop(['index','startDt','endDt'],axis=1, inplace=True)

# 설문지 데이터 + 수면 데이터 (이전 논문 데이터)
data=pd.read_csv('./data/last.csv')
data=data.drop(['Unnamed: 0','endInput'],axis=1)

data['total_sleep_time']=data['lightsleepduration']+data['deepsleepduration']+data['remsleepduration']

# 시간 파생변수 생성
data['startDt']=data['startDt'].astype('str')
data['startDt_year']=data['startDt'].apply(lambda x: x.split('-')[0]).astype('int')
data['startDt_month']=data['startDt'].apply(lambda x: x.split('-')[1]).astype('int')
data['startDt_day']=data['startDt'].apply(lambda x: x.split('-')[2].split(' ')[0]).astype('int')
data['startDt_weekend']=data['startDt'].apply(lambda x: dt.datetime.strptime(x.split(' ')[0], "%Y-%m-%d").weekday()).astype('int')

data['endDt']=data['endDt'].astype('str')
data['endDt_year']=data['endDt'].apply(lambda x: x.split('-')[0]).astype('int')
data['endDt_month']=data['endDt'].apply(lambda x: x.split('-')[1]).astype('int')
data['endDt_day']=data['endDt'].apply(lambda x: x.split('-')[2].split(' ')[0]).astype('int')
data['endDt_weekend']=data['endDt'].apply(lambda x: dt.datetime.strptime(x.split(' ')[0], "%Y-%m-%d").weekday()).astype('int')

data['lastUpdate']=data['lastUpdate'].astype('str')
data['lastUpdate_year']=data['lastUpdate'].apply(lambda x: x.split('-')[0]).astype('int')
data['lastUpdate_month']=data['lastUpdate'].apply(lambda x: x.split('-')[1]).astype('int')
data['lastUpdate_day']=data['lastUpdate'].apply(lambda x: x.split('-')[2].split(' ')[0]).astype('int')
data['lastUpdate_weekend']=data['lastUpdate'].apply(lambda x: dt.datetime.strptime(x.split(' ')[0], "%Y-%m-%d").weekday()).astype('int')

# 나이, 성별, 몸무게, 키,bmi 지수 파생변수 생성
for i in data['userId'].unique():
    age=data_info.loc[data_info['userId']==i,'age'].values[0]
    data.loc[data['userId']==i,'age']=age
    gender=data_info.loc[data_info['userId']==i,'gender'].values[0]
    data.loc[data['userId']==i,'gender']=gender
    height=data_info.loc[data_info['userId']==i,'height'].values[0]
    data.loc[data['userId']==i,'height']=height
    weight=data_info.loc[data_info['userId']==i,'weight'].values[0]
    data.loc[data['userId']==i,'weight']=weight
data['height_double']=data['height'].apply(lambda x:(x**2)*0.0001)
data['age']=pd.cut(data["age"], bins = 5, labels = [0, 1, 2, 3, 4]) # 나이 범주화
data['gender']=data['gender'].apply(lambda x : 1 if x=='M' else 0) #성별 이진화
data['bmi'] = data.apply(lambda row: math.floor((row['weight'] / row['height_double']) * 100) / 100, axis=1)

# BMI 수치에 따라 라벨링
def label_bmi(bmi):
    if bmi >=35: return 5 #3단계 비만
    elif 34.9 >= bmi >= 30: return 4 #2단계 비만
    elif 29.9 >= bmi >= 25: return 3 #1단계 비만
    elif 24.9 >= bmi >= 23: return 2 # 비만전단계
    elif 22.9 >= bmi >= 18.5: return 1 # 정상
    elif 18.5 > bmi: return 0 # 저체중
data['bmi_label'] = data['bmi'].apply(label_bmi)

## 범주형 변수 수치화
category=['age','sleepProblem','dream','startDt_day','endDt_day','lastUpdate_day','startDt_month','startDt_weekend', 'endDt_month','endDt_weekend','lastUpdate_month','lastUpdate_weekend']
data=pd.get_dummies(data=data,columns=category)    

## 데이터타입이 object인 컬럼 제거
data.drop(['alcohol','caffeine','startDt', 'endDt','lastUpdate','startDt_year','endDt_year','lastUpdate_year'],axis=1,inplace=True)

In [ ]:
## 데이터 1차 저장
data.to_csv('./2020_2th_paper_data_v2.csv')

# 활동 데이터와 병합

In [ ]:
busan=pd.DataFrame()
data=pd.read_csv('./2020_2th_paper_data_v2.csv').drop('Unnamed: 0',axis=1)
DIRNAME =sorted(os.listdir('./data/action_file'))
for i in range(len(DIRNAME)):
    userid=DIRNAME[i].split('.')[0]
    base_dir = './data/action_file/'+DIRNAME[i]
    label_data = pd.read_csv(base_dir)
    label_data=label_data[['date','action','actionOption']]
    datelist = sorted(label_data['date'].unique())
    col=['sleep', 'personal_care', 'work', 'study', 'household', 'care_housemem','recreation_media', 'entertainment', 'outdoor_act','hobby', 'recreation_etc' , 'shop', 'community_interaction', 'travel', 'meal', 'socialising']
    col2=[111, 112, 121, 122, 131, 132, 133, 134, 211, 212, 213, 22, 311, 312, 313, 314, 321, 322, 41, 42, 43, 44, 45, 46, 51, 52, 53, 54, 55, 56, 57, 81, 82, 83, 84, 85, 86,
                87, 61, 62, 63, 64, 711, 712, 713, 721, 722, 723, 724, 725, 741, 742, 743, 744, 745, 746, 751, 752, 753, 754, 755, 756, 761, 762, 763, 764, 791, 792, 793, 91, 92]
    raw1=pd.DataFrame(data=[[0]*len(col)],columns=col)
    raw2=pd.DataFrame(data=[[0]*len(col2)],columns=col2)
    for j in datelist:
        df=data[(data['userId']==userid)&(data['date']==j)]
        if len(df)==0:
            continue
        else:
            df2=label_data[(label_data['date']==j)]
            for z in df2['actionOption'].unique():
                df3=df2[df2['actionOption']==z].iloc[0,:]
                if df3['action']=='cargiving': raw1['care_housemem']=1
                elif df3['action']=='snack': raw1['meal']=1
                elif df3['action']=='commute': raw1['travel']=1
                elif df3['action']=='regular activity': raw1['community_interaction']=1
                elif df3['action']=='free time': raw1['recreation_etc']=1
                elif df3['action']=='sports': raw1['outdoor_act']=1
                elif df3['action'] in col: raw1[df3['action']]=1
                raw2[df3['actionOption']]=1
            collaboration=pd.concat([raw1,raw2],axis=1)
            if len(df)==1:
                stop=pd.concat([df.reset_index().drop('index',axis=1),collaboration],axis=1)
            elif len(df)==2:
                stop1=pd.concat([pd.DataFrame(df.iloc[0]).T.reset_index().drop('index',axis=1),collaboration],axis=1)
                stop2=pd.concat([pd.DataFrame(df.iloc[1]).T.reset_index().drop('index',axis=1),collaboration],axis=1)
                stop=pd.concat([stop1,stop2])
            busan=pd.concat([busan,stop],axis=0)

In [13]:
# 데이터 최종 저장
busan.to_csv('2020_2th_paper_data_plus_activity_v2.csv')

# 